In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [ ]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [ ]:
import sys
sys.path.insert(0, 'drive/DeepLearning')

In [ ]:
!pip install -q keras

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [ ]:
!cat /proc/meminfo

In [ ]:
!cat /proc/cpuinfo

In [15]:
# Load LSTM network and generate text
import sys
import string
import pandas as pd
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils


from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense,Dropout,Concatenate,Bidirectional
import numpy as np
import io

In [16]:
batch_size = 100  # Batch size for training.
epochs = 200  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 2000  # Number of samples to train on

In [17]:
wrongDataset = "wrongDataset.csv"
rightDataset = "rightDataset.csv"
wrong_Text = pd.read_csv(wrongDataset , delimiter='\n', header=None)
right_Text = pd.read_csv(rightDataset , delimiter='\n', header=None)

In [18]:
# create mapping of unique chars to integers, and a reverse mapping
wrongSentences = []
wrong_characters = set()
for i in wrong_Text.values[: min(num_samples, len(wrong_Text.values) - 1)]:
    tempSentence = ''.join(i)
    wrongSentences.append(tempSentence)
    for char in tempSentence:
        if char not in wrong_characters:
            wrong_characters.add(char)
        
rightSentences = []
right_characters = set()
for i in right_Text.values[: min(num_samples, len(right_Text.values) - 1)]:
    tempSentence = '\t' + ''.join(i) + '\n'
    rightSentences.append(tempSentence)
    for char in tempSentence:
        if char not in right_characters:
            right_characters.add(char)

In [19]:
wrong_characters = sorted(list(wrong_characters))
right_characters = sorted(list(right_characters))
num_encoder_tokens = len(wrong_characters)
num_decoder_tokens = len(right_characters)
max_encoder_seq_length = max([len(txt) for txt in wrongSentences])
max_decoder_seq_length = max([len(txt) for txt in rightSentences])

In [20]:
print('Number of Wrong Samples:', len(wrongSentences))
print('Number of Right Samples:', len(rightSentences))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of Wrong Samples: 2000
Number of Right Samples: 2000
Number of unique input tokens: 111
Number of unique output tokens: 113
Max sequence length for inputs: 852
Max sequence length for outputs: 856


In [21]:
wrong_token_index = dict(
    [(char, i) for i, char in enumerate(wrong_characters)])
right_token_index = dict(
    [(char, i) for i, char in enumerate(right_characters)])


In [22]:
encoder_input_data = np.zeros((len(wrongSentences), max_encoder_seq_length, num_encoder_tokens), dtype='float32')


In [23]:
decoder_input_data = np.zeros((len(wrongSentences), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

In [24]:
decoder_output_data = np.zeros((len(rightSentences), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

In [25]:
for i, (wrong_text, right_text) in enumerate(zip(wrongSentences, rightSentences)):
    for t, char in enumerate(wrong_text):
        encoder_input_data[i, t, wrong_token_index[char]] = 1.
    for t, char in enumerate(right_text):
        # decoder_output_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, right_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_output_data[i, t - 1, right_token_index[char]] = 1.

In [26]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = Bidirectional(LSTM(latent_dim, return_state=True))
encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder(encoder_inputs)
state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])
encoder_states = [state_h, state_c]


In [27]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))    
decoder_lstm = LSTM(latent_dim*2, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_output_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)



Train on 1600 samples, validate on 400 samples
Epoch 1/200


In [ ]:
model.save('s2s_2000_BiLSTM.h5')
model.summary()

In [ ]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

In [ ]:
decoder_state_input_h = Input(shape=(latent_dim*2,))
decoder_state_input_c = Input(shape=(latent_dim*2,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [ ]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in wrong_token_index.items())
reverse_right_char_index = dict(
    (i, char) for char, i in right_token_index.items())

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, right_token_index['\t']] = 1.
    
    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)
        
        # Sample a token
        sampled_right_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_right_char_index[sampled_right_index]
        decoded_sentence += sampled_char
        
        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_right_index] = 1.
        
        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
for seq_index in range(int(num_samples * 0.2)):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', wrongSentences[seq_index])
    print('Decoded sentence:', decoded_sentence)